In [15]:
import pandas as pd
from sqlalchemy import create_engine

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_escolas = pd.read_csv('fontes/dataset_escolas.csv')
df_estudantes = pd.read_csv('fontes/dataset_estudantes.csv')

,ID_Estudante,Nome_Estudante,Genero,Serie,Nome_Escola,Nota_Redacao,Nota_Matematica
0,0,Kevin Bradley,M,6,Escola A,66,79
1,1,Paul Smith,M,9,Escola A,94,61
2,2,John Rodriguez,M,9,Escola A,90,60
3,3,Oliver Scott,M,9,Escola A,67,58
4,4,William Ray,F,6,Escola A,97,84
...,...,...,...,...,...,...,...
39155,39155,John Brooks,M,7,Escola O,92,98
39156,39156,Stephanie Contreras,F,8,Escola O,79,95
39157,39157,Kristen Gonzalez,F,6,Escola O,79,94
39158,39158,Kari Holloway,F,7,Escola O,87,90


In [8]:
df_escolas.isnull().sum()

ID_Escola          0
Nome_Escola        0
Tipo_Escola        0
Numero_Alunos      0
Orcamento_Anual    0
dtype: int64

In [9]:
df_escolas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID_Escola        15 non-null     int64 
 1   Nome_Escola      15 non-null     object
 2   Tipo_Escola      15 non-null     object
 3   Numero_Alunos    15 non-null     int64 
 4   Orcamento_Anual  15 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 732.0+ bytes


In [11]:
#para nomes de colunas diferentes:
#lefton='coluna da primeira tabela', righton='coluna da segunda tabela'
#indicator=True - verifica correspondencia do dado nas duas tabelas
df_merged = pd.merge(df_estudantes, df_escolas, on='Nome_Escola', how='left')
df_merged

,ID_Estudante,Nome_Estudante,Genero,Serie,Nome_Escola,Nota_Redacao,Nota_Matematica,ID_Escola,Tipo_Escola,Numero_Alunos,Orcamento_Anual
0,0,Kevin Bradley,M,6,Escola A,66,79,0,Publica,2917,1910635
1,1,Paul Smith,M,9,Escola A,94,61,0,Publica,2917,1910635
2,2,John Rodriguez,M,9,Escola A,90,60,0,Publica,2917,1910635
3,3,Oliver Scott,M,9,Escola A,67,58,0,Publica,2917,1910635
4,4,William Ray,F,6,Escola A,97,84,0,Publica,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39155,39155,John Brooks,M,7,Escola O,92,98,14,Particular,1635,1043130
39156,39156,Stephanie Contreras,F,8,Escola O,79,95,14,Particular,1635,1043130
39157,39157,Kristen Gonzalez,F,6,Escola O,79,94,14,Particular,1635,1043130
39158,39158,Kari Holloway,F,7,Escola O,87,90,14,Particular,1635,1043130


In [16]:
media_escola = df_merged.groupby('Tipo_Escola')[['Nota_Redacao','Nota_Matematica']].mean()
media_escola

,Nota_Redacao,Nota_Matematica
Tipo_Escola,,
Particular,83.897735,83.408404
Publica,80.962485,76.987026


In [21]:
df_merged.to_csv('fontes/escolas_merged.csv', index=False)

df_merged.to_excel('fontes/escolas_merged.xlsx', index=False)

df_merged.to_json('fontes/escolas_merged.json', orient='records', indent=2)

In [23]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39160 entries, 0 to 39159
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID_Estudante     39160 non-null  int64 
 1   Nome_Estudante   39160 non-null  object
 2   Genero           39160 non-null  object
 3   Serie            39160 non-null  int64 
 4   Nome_Escola      39160 non-null  object
 5   Nota_Redacao     39160 non-null  int64 
 6   Nota_Matematica  39160 non-null  int64 
 7   ID_Escola        39160 non-null  int64 
 8   Tipo_Escola      39160 non-null  object
 9   Numero_Alunos    39160 non-null  int64 
 10  Orcamento_Anual  39160 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 3.3+ MB


In [26]:
pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
usuario = 'postgres'
senha = 'sa'
host = 'localhost'
database = 'joabe_db'
port = '5432'

engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{port}/{database}')

In [29]:
df_merged.to_sql('estudantes_escolas', engine, if_exists='replace', index=False)

160

In [30]:
df_db = pd.read_sql('select * from estudantes_escolas', engine)
df_db

,ID_Estudante,Nome_Estudante,Genero,Serie,Nome_Escola,Nota_Redacao,Nota_Matematica,ID_Escola,Tipo_Escola,Numero_Alunos,Orcamento_Anual
0,0,Kevin Bradley,M,6,Escola A,66,79,0,Publica,2917,1910635
1,1,Paul Smith,M,9,Escola A,94,61,0,Publica,2917,1910635
2,2,John Rodriguez,M,9,Escola A,90,60,0,Publica,2917,1910635
3,3,Oliver Scott,M,9,Escola A,67,58,0,Publica,2917,1910635
4,4,William Ray,F,6,Escola A,97,84,0,Publica,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39155,39155,John Brooks,M,7,Escola O,92,98,14,Particular,1635,1043130
39156,39156,Stephanie Contreras,F,8,Escola O,79,95,14,Particular,1635,1043130
39157,39157,Kristen Gonzalez,F,6,Escola O,79,94,14,Particular,1635,1043130
39158,39158,Kari Holloway,F,7,Escola O,87,90,14,Particular,1635,1043130
